In [1]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import cv2
import os

In [2]:
prototxtPath = 'deploy.prototxt.txt'
weightsPath = 'res10_300x300_ssd_iter_140000.caffemodel'

faceNet=cv2.dnn.readNet(prototxtPath,weightsPath)

In [3]:
maskNet=load_model('mobile-net-v2-my-trained-model.model')

In [4]:
ori_image=cv2.imread('mynomaskimg.jpg')

In [5]:
ori_image.shape

(720, 1280, 3)

In [6]:
(h,w) = ori_image.shape[0:2]
h,w

(720, 1280)

In [7]:
# blobFromImage(image, scalefactor=1.0, size, mean, swapRB=True) CV2 read image is BGR, there swap R and B.

blob=cv2.dnn.blobFromImage(ori_image, 1.0, (300,300), (104.0,177.0,123.0))

In [8]:
blob.shape

(1, 3, 300, 300)

In [9]:
faceNet.setInput(blob)
detections=faceNet.forward()

In [10]:
print('detections.shape -> ', detections.shape)
print('confidence -> ', detections[0,0,0,2])
print('since "confidence > 0.5" i.e. face is detected we extract dimensions -> ',detections[0,0,0,3:7] )

detections.shape ->  (1, 1, 200, 7)
confidence ->  0.9999058
since "confidence > 0.5" i.e. face is detected we extract dimensions ->  [0.46222767 0.21717578 0.67306066 0.7583098 ]


In [11]:
image = ori_image.copy()

#loop over the detections
for i in range(0,detections.shape[2]):
    confidence=detections[0,0,i,2]
    
    
    if confidence>0.5:
        #we need the X,Y coordinates
        box=detections[0,0,i,3:7]*np.array([w,h,w,h])
        (startX,startY,endX,endY)=box.astype('int')
        
        #ensure the bounding boxes fall within the dimensions of the frame
        (startX,startY)=(max(0,startX),max(0,startY))
        (endX,endY)=(min(w-1,endX), min(h-1,endY))
        
        
        #extract the face ROI, convert it from BGR to RGB channel, resize it to 224,224 and preprocess it
        face = image[startY:endY, startX:endX]
        face = cv2.cvtColor(face,cv2.COLOR_BGR2RGB)
        face = cv2.resize(face,(224,224))
        face = img_to_array(face)
        face = preprocess_input(face)
        face = np.expand_dims(face,axis=0)
        
        (mask,withoutMask) = maskNet.predict(face)[0]
        
        #determine the class label and color we will use to draw the bounding box and text
        label='Mask' if mask>withoutMask else 'No Mask'
        color=(0,255,0) if label=='Mask' else (0,0,255)
        
        #include the probability in the label
        label="{}: {:.2f}%".format(label,max(mask,withoutMask)*100)
        
        #display the label and bounding boxes
        cv2.putText(image,label,(startX,startY-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,color,2)
        cv2.rectangle(image,(startX,startY),(endX,endY),color,2)
        
        
        
cv2.imshow("Output",image)
cv2.waitKey(0)
cv2.destroyAllWindows()